# From OCS Documentation to Executable Notebook  

The idea is to easily and straightforwardly convert static OCS request (implicit or not) like: 

![](https://academichub.blob.core.windows.net/images/ocs-dataview-create-quickstart.png)

into its equivalent notebook version which can be executed: 

![](https://academichub.blob.core.windows.net/images/ocs-dataview-create-quickstart-cell.png)

**Moreover with a free service like [Binder](https://mybinder.org/) it's possible to add a simple Binder icon to our documentation like this one :
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/osisoft-academic/HackDavis2020/master?filepath=DataView_QS_Example_Scenario.ipynb)
(you can try it, it's live)** 

**and allow people to "execute" the documentation without having to install anything to try it.**

#### This is a proof-of-concept using the [Data View - Example Scenario](https://ocs-docs.osisoft.com/Documentation/DataViews/Example_Scenario.html) as a starting point since it's necessary step for the subsequent [Data View - Quick Start](https://ocs-docs.osisoft.com/Documentation/DataViews/Quick_Start_Define_a_Data_View.html) 
---
**CHANGELOG**

* Feb 20, 2020 - using official ocs_sample_library_preview version 0.1.3rc0 
* Feb 10, 2020 - initial version
---

## Install modified OCS sample python library

Source: [https://github.com/osisoft-academic/OSI-Samples-OCS](https://github.com/osisoft-academic/OSI-Samples-OCS)

Modfied initially for [Locust](https://locust.io), now also support the magic cell command `%ocs_request` allowing cut-and-paste-execute from Data View OCS documentation

In [1]:
!pip install ocs-sample-library-preview==0.1.3.rc0

## Standard imports for OCS python application 

In [2]:
from ocs_sample_library_preview import OCSClient, SdsStream

import configparser
import json
import io
import random

## Please enter your OCS credentials -- should have write access to the target namespace 

In [3]:
config_text = u"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = Samples

[Access]
Resource = https://dat-b.osisoft.com
Tenant = 65292b6c-ec16-414a-b583-ce7ae04046d4
ApiVersion = v1-preview

[Credentials] 
ClientId = *********************************
ClientSecret = *******************************
"""

In [4]:
config = configparser.ConfigParser(allow_no_value=True)
config.read_file(io.StringIO(config_text))

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

tenantId = config.get("Access", "Tenant")
namespaceId = config.get("Configurations", "Namespace")
print(f"tenantId: {tenantId}, namespaceId: '{namespaceId}'")

tenantId: 65292b6c-ec16-414a-b583-ce7ae04046d4, namespaceId: 'Samples'


## Define the `ocs_request` magic cell command 

This is what allows to simply put `%%ocs_request` as the first line of a cell, then cut-and-paste from OCS docs to execute the request live. First usage is two cells below. 

In [5]:
from IPython.core.magic import (
    register_line_magic,
    register_cell_magic,
    register_line_cell_magic,
)

# map doc values to their Python equivalent
null = ""
true = True

# the magic cell command itself
@register_cell_magic
def ocs_request(_, req):  # req contains the cell text without %%ocs_request
    sl = req.split("\n")  # get list of strings, one per line
    sl0 = sl[0].split(" ")  # first line always HTTP_VERB OCS_URL_TO_RESOLVE
    verb = sl0[0]
    url = sl0[1].format(tenantId=tenantId, namespaceId=namespaceId)  # resolve URL
    full_url = ocs_client._OCSClient__baseClient.uri + url
    if verb.lower() != "get" and verb.lower() != "delete":
        js = json.loads(
            "".join(sl[1:])
        )  # text on remaining lines is a JSON to transfer
        response = ocs_client._OCSClient__baseClient.request(verb, full_url, json=js)
        print("==>", verb, full_url, f"[{response.status_code}]\n", js)
    else:
        response = ocs_client._OCSClient__baseClient.request(verb, full_url)
        print(
            "==> GET URL=",
            verb,
            full_url,
            f"[{response.status_code}]",
            "\n  ",
            response.json(),
        )
    try:
        return response.status_code, response.json()
    except:
        return response.status_code, response.content

# The rest is copied from OCS Data View / Example Scenario

Source: [https://ocs-docs.osisoft.com/Documentation/DataViews/Example_Scenario.html](https://ocs-docs.osisoft.com/Documentation/DataViews/Example_Scenario.html)

**Goal: have an notebook-executable version of the doc so people can experiment** 

---
---

# Example Scenario

This section uses example streams to illustrate data view concepts. The streams are of three types:

* docs-pi-inverter | simulates solar power inverter data collected via PI to OCS
* docs-omf-weather-gen1 | simulates weather data collected from an OMF source
* docs-omf-weather-gen2 | simulates weather data collected from an OMF source, but with some additional/renamed properties
* Each solar inverter is associated with some physical location. There is a weather stream for each location.

Data views are made to bring order to a tangle of data streams. Despite appearing complex and verbose, this scenario is likely much simpler than your real-world data. This scenario illustrates how data views can render real-world data consumable for data science.

## Solar Inverter streams
These streams represent values collected via PI to OCS, originally collected by a PI System. Each stream value contains a timestamp (Timestamp) and a measurement (Value).

### Type

In [6]:
%%ocs_request
POST /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Types/docs-pi-inverter
  {
    "Id": "docs-pi-inverter",
    "Name": "Inverter",
    "SdsTypeCode": "Object",
    "Properties": [
      {
        "Id": "Timestamp",
        "Name": "Timestamp",
        "IsKey": true,
        "SdsType": { "SdsTypeCode": "DateTime" }
      },
      {
        "Id": "Value",
        "Name": "Value",
        "SdsType": { "SdsTypeCode": "Double" }
      }
    ]
  }

==> POST https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Types/docs-pi-inverter [200]
 {'Id': 'docs-pi-inverter', 'Name': 'Inverter', 'SdsTypeCode': 'Object', 'Properties': [{'Id': 'Timestamp', 'Name': 'Timestamp', 'IsKey': True, 'SdsType': {'SdsTypeCode': 'DateTime'}}, {'Id': 'Value', 'Name': 'Value', 'SdsType': {'SdsTypeCode': 'Double'}}]}


(200,
 {'Id': 'docs-pi-inverter',
  'Name': 'Inverter',
  'SdsTypeCode': 1,
  'Properties': [{'Id': 'Timestamp',
    'Name': 'Timestamp',
    'IsKey': True,
    'SdsType': {'SdsTypeCode': 16}},
   {'Id': 'Value', 'Name': 'Value', 'SdsType': {'SdsTypeCode': 14}}]})

### Stream IDs
---
16 inverter streams are defined:

    "BILT.Meter.Primary.Inverter.0.PwrIn",
    "BILT.Meter.Primary.Inverter.0.PwrOut",
    "BILT.Meter.Primary.Inverter.1.PwrIn",
    "BILT.Meter.Primary.Inverter.1.PwrOut",
    "BILT.Meter.Primary.Inverter.2.PwrIn",
    "BILT.Meter.Primary.Inverter.2.PwrOut",
    "BILT.Meter.Secondary.Inverter.0.PwrIn",
    "BILT.Meter.Secondary.Inverter.0.PwrOut",
    "ROSE.Meter.Primary.Inverter.0.PwrIn",
    "ROSE.Meter.Primary.Inverter.0.PwrOut",
    "ROSE.Meter.Primary.Inverter.1.PwrIn",
    "ROSE.Meter.Primary.Inverter.1.PwrOut",
    "WINT.Meter.Primary.Inverter.0.PwrIn",
    "WINT.Meter.Primary.Inverter.0.PwrOut",
    "WINT.Meter.Secondary.Inverter.0.PwrIn",
    "WINT.Meter.Secondary.Inverter.0.PwrOut",


### Tags and Metadata
---
Each inverter stream has some descriptive tags assigned, and metadata key-values describing its function.
<pre>
Tags: some subset of [ "Commercial", "Residential", "Critical Asset" ]
Metadata: { 
            "Site" : one of ( "Biltmore" | "Rosecliff" | "Winterthur" ),
            "Measurement": one of ( "Power In" | "Power Out" ),
            "Meter": one of ( "Primary", "Secondary" ),
            "Inverter": one of ( "0", "1", "2" )
            "Nominal Power MW": "1.21"
          }
</pre>

In [7]:
site_map = {"BILT": "Biltmore", "ROSE": "Rosecliff", "WINT": "Winterthur"}
measure_map = {"PwrIn": "Power In", "PwrOut": "Power Out"}
meter_kind = ["Primary", "Secondary"]
inverter_num = ["0", "1", "2"]
customer_tags = ["Commercial", "Residential"]
critical = "Critical Asset"

In [8]:
for site in site_map.keys():
    for meter in meter_kind:
        for inverter in inverter_num:
            for measurement in measure_map.keys():
                if site == "BILT" and meter == "Secondary" and int(inverter) >= 1:
                    continue
                if site == "ROSE" and (meter == "Secondary" or int(inverter) >= 2):
                    continue
                if site == "WINT" and int(inverter) >= 1:
                    continue
                stream_id = f"{site}.Meter.{meter}.Inverter.{inverter}.{measurement}"
                stream = SdsStream(
                    id=stream_id, name=stream_id, typeId="docs-pi-inverter"
                )
                #
                # Create a new OCS stream
                #
                stream_req = (
                    f"POST /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Streams/{stream_id}\n"
                    + str(stream.toJson())
                )
                ocs_request(None, stream_req)
                #
                # Update stream metadata
                #
                metadata = {
                    "Site": site_map[site],
                    "Measurement": measure_map[measurement],
                    "Meter": meter,
                    "Inverter": inverter,
                }
                metadata_req = (
                    f"PUT /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Streams/{stream_id}/Metadata\n"
                    + json.dumps(metadata)
                )
                ocs_request(None, metadata_req)
                #
                # Update stream tags
                #
                tags = [random.choice(customer_tags)]
                if random.randint(0, 1) == 1:
                    tags += [critical]
                tags_req = (
                    f"PUT /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Streams/{stream_id}/Tags\n"
                    + json.dumps(tags)
                )
                ocs_request(None, tags_req)

==> POST https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Streams/BILT.Meter.Primary.Inverter.0.PwrIn [200]
 {'Id': 'BILT.Meter.Primary.Inverter.0.PwrIn', 'TypeId': 'docs-pi-inverter', 'Name': 'BILT.Meter.Primary.Inverter.0.PwrIn', 'Description': None, 'InterpolationMode': None, 'ExtrapolationMode': None}
==> PUT https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Streams/BILT.Meter.Primary.Inverter.0.PwrIn/Metadata [200]
 {'Site': 'Biltmore', 'Measurement': 'Power In', 'Meter': 'Primary', 'Inverter': '0'}
==> PUT https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Streams/BILT.Meter.Primary.Inverter.0.PwrIn/Tags [200]
 ['Commercial']
==> POST https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Streams/BILT.Meter.Primary.Inverter.0.PwrOut [200]
 {'Id': 'BILT.Meter.Primary.In

---
---
# TODO: create streams, add sample data to streams 

---
---

## Weather Streams
These streams simulate data collected via OMF from a weather station. There are two "generations" represented: Gen2 adds one property and renames another, as compared to Gen1.

### Tags and Metadata
---
Each weather stream has some descriptive tags assigned, and one metadata key-value indicating its Site. These are the same sites with which the inverters are associated.

    Tags: some subset of [ "Weather", "Low Resolution", "High Resolution", "Gen1", "Gen2" ]
    Metadata: { "Site" : one of ( "Biltmore" | "Rosecliff" | "Winterthur" ) }
    
## Weather, Generation 1

### Type

In [9]:
%%ocs_request
POST /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Types/docs-omf-weather-gen1
  {
    "Id": "docs-omf-weather-gen1",
    "Name": "WeatherGen1",
    "SdsTypeCode": "Object",
    "Properties": [
      {
        "Id": "Timestamp",
        "Name": "Timestamp",
        "IsKey": true,
        "SdsType": { "SdsTypeCode": "DateTime" }
      },
      {
        "Id": "SolarRadiation",
        "Name": "Solar Radiation",
        "SdsType": { "SdsTypeCode": "Int32" }
      },
      {
        "Id": "Temperature",
        "Name": "Temperature",
        "SdsType": { "SdsTypeCode": "Double" }
      }
    ]
  }

==> POST https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Types/docs-omf-weather-gen1 [200]
 {'Id': 'docs-omf-weather-gen1', 'Name': 'WeatherGen1', 'SdsTypeCode': 'Object', 'Properties': [{'Id': 'Timestamp', 'Name': 'Timestamp', 'IsKey': True, 'SdsType': {'SdsTypeCode': 'DateTime'}}, {'Id': 'SolarRadiation', 'Name': 'Solar Radiation', 'SdsType': {'SdsTypeCode': 'Int32'}}, {'Id': 'Temperature', 'Name': 'Temperature', 'SdsType': {'SdsTypeCode': 'Double'}}]}


(200,
 {'Id': 'docs-omf-weather-gen1',
  'Name': 'WeatherGen1',
  'SdsTypeCode': 1,
  'Properties': [{'Id': 'Timestamp',
    'Name': 'Timestamp',
    'IsKey': True,
    'SdsType': {'SdsTypeCode': 16}},
   {'Id': 'SolarRadiation',
    'Name': 'Solar Radiation',
    'SdsType': {'SdsTypeCode': 9}},
   {'Id': 'Temperature',
    'Name': 'Temperature',
    'SdsType': {'SdsTypeCode': 14}}]})

### Stream IDs
---
Two streams of this type are defined:

    "WS_BILT",
    "WS_ROSE",

## Weather, Generation 2

The latest and greatest Gen2 weather stations report a new property, Cloud Cover, but also rename Temperature to Ambient Temperature. They are otherwise similar to Gen1.

### Type

In [10]:
%%ocs_request
POST /api/v1-preview/Tenants/{tenantId}/Namespaces/{namespaceId}/Types/docs-omf-weather-gen2
  {
    "Id": "docs-omf-weather-gen2",
    "Name": "WeatherGen2",
    "SdsTypeCode": "Object",
    "Properties": [
      {
        "Id": "Timestamp",
        "Name": "Timestamp",
        "IsKey": true,
        "SdsType": { "SdsTypeCode": "DateTime" }
      },
      {
        "Id": "SolarRadiation",
        "Name": "Solar Radiation",
        "SdsType": { "SdsTypeCode": "Int32" }
      },
      {
        "Id": "AmbientTemperature",
        "Name": "Ambient Temperature",
        "SdsType": { "SdsTypeCode": "Double" }
      },
      {
        "Id": "CloudCover",
        "Name": "Cloud Cover",
        "SdsType": { "SdsTypeCode": "Int32" }
      }
    ]
  }

==> POST https://dat-b.osisoft.com/api/v1-preview/Tenants/65292b6c-ec16-414a-b583-ce7ae04046d4/Namespaces/Samples/Types/docs-omf-weather-gen2 [200]
 {'Id': 'docs-omf-weather-gen2', 'Name': 'WeatherGen2', 'SdsTypeCode': 'Object', 'Properties': [{'Id': 'Timestamp', 'Name': 'Timestamp', 'IsKey': True, 'SdsType': {'SdsTypeCode': 'DateTime'}}, {'Id': 'SolarRadiation', 'Name': 'Solar Radiation', 'SdsType': {'SdsTypeCode': 'Int32'}}, {'Id': 'AmbientTemperature', 'Name': 'Ambient Temperature', 'SdsType': {'SdsTypeCode': 'Double'}}, {'Id': 'CloudCover', 'Name': 'Cloud Cover', 'SdsType': {'SdsTypeCode': 'Int32'}}]}


(200,
 {'Id': 'docs-omf-weather-gen2',
  'Name': 'WeatherGen2',
  'SdsTypeCode': 1,
  'Properties': [{'Id': 'Timestamp',
    'Name': 'Timestamp',
    'IsKey': True,
    'SdsType': {'SdsTypeCode': 16}},
   {'Id': 'SolarRadiation',
    'Name': 'Solar Radiation',
    'SdsType': {'SdsTypeCode': 9}},
   {'Id': 'AmbientTemperature',
    'Name': 'Ambient Temperature',
    'SdsType': {'SdsTypeCode': 14}},
   {'Id': 'CloudCover',
    'Name': 'Cloud Cover',
    'SdsType': {'SdsTypeCode': 9}}]})

### Stream IDs
---
One stream of this type is defined:

    "WS_WINT"